In [18]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics
#from sklearn import cross_validation
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.feature_extraction import stop_words
from sklearn.metrics import f1_score
from nltk.stem.snowball import SnowballStemmer
from sklearn.utils import resample
from sklearn.metrics import precision_recall_curve
import string, re

%matplotlib inline

In [20]:
pd.set_option('max_colwidth', 140)
data[100:200]

,question_text,target
qid,,
00043c2c68e74328c456,"What do physicists, mathematicians, computer scientists and philosophers think of David Deutsch's 'Constructor Theory'?",0
00043d911af1cfbdb5f3,Why are old scriptures from eastern cultures appear lost in the current culture?,0
000441059c27001eb255,"Can I know my I.Q, even if I hate numbers?",0
00045f3b9fcb27975e26,How can I really make up my mind and get rid of my bad habits like procrastination?,0
00046512985c0996339e,Was there any relationship between Napoleon and Ali Pasha of Tepelene?,0
000467723d6f04760035,Where are presynaptic neurons found?,0
000477ab08d14b6a047d,What ways will a narcissist mother punish her child for going no contact if child goes back to contact with her?,0
000485e6dd4b149fe051,Can I start freelancing after finishing Udacity's Android basic nanodegree?,0
000488ff2dbaa802b4d9,What is the reason why we really need Bitcoin?,0


In [3]:
#data_small = data.sample(100000, replace = False)

In [4]:
# stop =  stop_words.ENGLISH_STOP_WORDS
# stemmer = SnowballStemmer('english')
# data_stemmed = data.copy()
# data_stemmed['question_text'] = [' '.join([stemmer.stem(word) for word in text.split(' ')])
#           for text in data_stemmed.question_text]

In [49]:
def data_prep_training(dataset, training_split, test_split):
    train, test = train_test_split(dataset, train_size = training_split, test_size = test_split)
    train_downsampled = downsample(train)
    
    X_train = train_downsampled['question_text']
    Y_train = train_downsampled['target']
    X_test = test['question_text']
    Y_test = test['target']
    
    return(X_train, X_test, Y_train, Y_test)

def data_prep_training_nodownsample(dataset, training_split, test_split):
    train, test = train_test_split(dataset, train_size = training_split, test_size = test_split)
    #train_downsampled = downsample(train)
    
    X_train = train['question_text']
    Y_train = train['target']
    X_test = test['question_text']
    Y_test = test['target']
    
    return(X_train, X_test, Y_train, Y_test)

def data_prep(dataset, training_split, test_split):
    X = dataset['question_text']
    Y = dataset['target']
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = training_split, test_size = test_split)
    
    return(X_train, X_test, Y_train, Y_test)


def model_vectorize(data_used, vectorizer_type, binary_type, ngram, stop_word, model_type):
    X_train = data_used[0]
    X_test = data_used[1]
    Y_train = data_used[2]
    Y_test = data_used[3]
    
    vectorizer = vectorizer_type(binary = binary_type, stop_words = stop_word, ngram_range=ngram)
    vectorizer.fit(X_train)
    X_train_vectorized = vectorizer.transform(X_train)
    X_test_vectorized = vectorizer.transform(X_test)
    model = model_type
    model.fit(X_train_vectorized, Y_train)
    
    scores = model.predict_proba(X_test_vectorized)[:,1]
    precision, recall, thresholds = precision_recall_curve(Y_test, scores)
    precision, recall = precision[:-1], recall[:-1]
    fscores = 2*np.divide(np.multiply(precision, recall), np.add(precision, recall))
    max_fscore = np.nanmax(fscores)
    ind_max = fscores.argmax() #not sure this is working
    threshold_max = thresholds[ind_max]
    
    return(max_fscore, threshold_max)


def downsample(df):
    # Separate majority and minority classes
    df_majority = df[df.target==0]
    df_minority = df[df.target==1]

    # Downsample majority class
    df_majority_downsampled = resample(df_majority, 
                                     replace=False,    # sample without replacement
                                     n_samples=df_minority.shape[0],     # to match minority class
                                     random_state=123) # reproducible results

    # Combine minority class with downsampled majority class
    df_downsampled = pd.concat([df_majority_downsampled, df_minority])
    
    return (df_downsampled)

In [6]:
# downsample_data_regular = data_prep(downsample(data), .9, .1)
# downsample_data_stemmed = data_prep(downsample(data_stemmed), .9,.1)

In [7]:
# test = model_vectorize(downsample_data_regular, CountVectorizer, True, (1,2), None, LogisticRegression())

In [12]:
# fscores = []
# thresholds = []
# fscores.append(test[0])
# thresholds.append(test[1])
# metrics = pd.DataFrame({'fscore': fscores, 'threshold': thresholds})
# metrics.sort_values(by = ['fscore'], ascending=False)

,fscore,threshold
0,0.897366,0.431035


In [13]:
# test2 = model_vectorize(downsample_data_regular, CountVectorizer, True, (1,2), None, BernoulliNB())

In [14]:
# fscores = []
# thresholds = []
# fscores.append(test2[0])
# thresholds.append(test2[1])
# metrics = pd.DataFrame({'fscore': fscores, 'threshold': thresholds})
# metrics.sort_values(by = ['fscore'], ascending=False)

,fscore,threshold
0,0.872824,0.041563


In [16]:
# test3 = model_vectorize(downsample_data_stemmed, CountVectorizer, True, (1,2), None, BernoulliNB())
# test4 = model_vectorize(downsample_data_regular, CountVectorizer, True, (1,3), None, BernoulliNB())
# test5 = model_vectorize(downsample_data_regular, CountVectorizer, True, (1,3), stop, BernoulliNB())
# test6 = model_vectorize(downsample_data_regular, TfidfVectorizer, False, (1,2), None, BernoulliNB())

In [17]:
# fscores = []
# thresholds = []
# for i in [test, test2, test3, test4, test5, test6]:
#     fscores.append(i[0])
#     thresholds.append(i[1])
# metrics = pd.DataFrame({'fscore': fscores, 'threshold': thresholds})
# metrics.sort_values(by = ['fscore'], ascending=False)

,fscore,threshold
0,0.897366,0.431035
4,0.878318,0.004500
1,0.872824,0.041563
5,0.872824,0.041563
3,0.872032,0.000451
2,0.871758,0.037112


In [27]:
# len(downsample_data_regular)
# #data.target.value_counts()
# #len(downsample_data_regular[0]), len(downsample_data_regular[1]), len(downsample_data_regular[2]), len(downsample_data_regular[3])

4

In [ ]:
# #create models
# dataset = [downsample_data_regular, downsample_data_stemmed]
# vectorizer_type = [CountVectorizer, TfidfVectorizer]
# binary_type = [True, False]
# ngram = [(1,2), (1,3), (1,4)]
# stop_word = [None, stop]
# model_type = [LogisticRegression(), BernoulliNB()]

# model_initialize = []
# for h in dataset:
#     for i in vectorizer_type:
#         for j in binary_type:
#             for k in ngram:
#                 for l in model_type:
#                     for m in stop_word:
#                         model_initialize.append(model_vectorize(h, i, j, k, m, l))
                    
# #create labels
# dataset_label = ['Downsample Full Regular Data', 'Downsample Full Stemmed Data']
# vectorizer_type_label = ['CountV', 'TFIDV']
# binary_type_label = ['T', 'F']
# ngram_label = [(1,2), (1,3), (1,4)]
# stop_word_label = ['None', 'english']
# model_type_label = ['LR', 'NB']

# labels = []
# for h in dataset_label:    
#     for i in vectorizer_type_label:
#         for j in binary_type_label:
#             for k in ngram_label:
#                 for l in model_type_label:
#                     for m in stop_word_label:
#                         label = '%s %s %s %s %s %s' %(h, i, j, k, m, l)
#                         labels.append(label)
                        
# fscores = []
# thresholds = []
# for i in model_initialize:
#     fscores.append(i[0])
#     thresholds.append(i[1])
# metrics = pd.DataFrame({'label': labels,'fscore': fscores, 'threshold': thresholds})
# metrics.sort_values(by = ['fscore'], ascending=False)

In [54]:
def run_model(train_data, test_data, vectorizer_type, binary_type, ngram, stop_word, model_type, max_t):
    X_train = train_data['question_text']
    Y_train = train_data['target']
    X_test = test_data['question_text']
    
    
    vectorizer = vectorizer_type(binary = binary_type, stop_words = stop_word, ngram_range=ngram)
    vectorizer.fit(X_train)
    X_train_vectorized = vectorizer.transform(X_train)
    X_test_vectorized = vectorizer.transform(X_test)
    model = model_type
    model.fit(X_train_vectorized, Y_train)
    
    predictions = model.predict_proba(X_test_vectorized)[:,1]
    max_threshold = max_t
    targets = []
    for i in predictions:
        if i<= max_threshold:
            targets.append(0)
        else:
            targets.append(1)
    
    
    
    return(predictions, targets)

In [69]:
downsampled = downsample(train_data)
test = run_model(downsampled, test_data, CountVectorizer, True, (1,2), None, LogisticRegression())

In [87]:
def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

def _get_groupings(groupings_dict):
    grouping_re = re.compile('(%s)' % '|'.join(groupings_dict.keys()))
    return groupings_dict, grouping_re


mispell_dict = {'colour':'color',
                'centre':'center',
                'didnt':'did not',
                'doesnt':'does not',
                'wouldnt':'would not',
                'isnt':'is not',
                'wouldnt':'would not',
                'shouldnt':'should not',
                'favourite':'favorite',
                'neighbour': 'neighbor',
                'humour': 'humor',
                'apologise': 'apologize',
                'travelling':'traveling',
                'counselling':'counseling',
                'recognise': 'recognize',
                'theatre':'theater',
                'cancelled':'canceled',
                'travelled': 'traveled',
                'offence': 'offense',
                'licence': 'license',
                'labour':'labor',
                'behaviour': 'behavior',
                'organisation':'organization',
                'wwii':'world war 2',
                'criticise':'criticize',
                }

groupings_dict = {'instagram': 'social_media',
                  'whatsapp': 'social_media',
                  'snapchat': 'social_media',
                  'facebook': 'social_media',
                  'wechat': 'social_media',
                  'groupme': 'social_media',
                  'iphone': 'cellphone',
                  'samsung galaxy': 'cellphone',
                  'google pixel': 'cellphone',
                  'president donald trump': 'political_person',
                  'donald trump': 'political_person',
                  'president trump': 'political_person',
                  'trump': 'political_person',
                  'president barack obama': 'political_person',
                  'president obama': 'political_person',
                  'barack obama': 'political_person',
                  'obama': 'political_person',
                  'george w bush': 'political_person',
                  'george trump': 'political_person',
                  'president bush': 'political_person',
                  'president trump': 'political_person',
                  'jfk': 'political_person',
                  'president nixon': 'political_person',
                  'nixon': 'political_person',
                  'democrats': 'political_party',
                  'democrat': 'political_party',
                  'republicans': 'political_party',
                  'democratic': 'political_party',
                  'republican': 'political_party',
                  'president trump': 'political_person',
                  'homosexual': 'lgbtq',
                  'heterosexual': 'lgbtq',
                  'homosexual': 'lgbtq', 
                  'bisexual': 'lgbtq', 
                  'transgender': 'lgbtq', 
                  'pansexual': 'lgbtq', 
                  'gen x': 'generation',
                  'gen y': 'generation',
                  'gen z': 'generation',
                  'millennials': 'generation',
                  'millennial': 'generation',
                  'milennials': 'generation',
                  'milennial': 'generation',
                  'milenials': 'generation',
                  'milenial': 'generation',
                  
    
}
mispellings, mispellings_re = _get_mispell(mispell_dict)
groupings, groupings_re = _get_groupings(groupings_dict)

def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]

    return mispellings_re.sub(replace, text)

def replace_typical_groupings(text):
    def replace(match):
        return groupings[match.group(0)]

    return groupings_re.sub(replace, text)





def clean(text):
    # Remove puncuation
    text = ''.join(ch for ch in text if ch not in string.punctuation)
    
    # Convert words to lower case and split them
    text = text.lower()

    # Clean the text
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"'m", " am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    
    text = re.sub('[0-9]{5,}', '#####', text)
    text = re.sub('[0-9]{4}', '####', text)
    text = re.sub('[0-9]{3}', '###', text)
    text = re.sub('[0-9]{2}', '##', text)
    text = re.sub('[0-9]{1}', '#', text)
    
    #Replace typical misspells
    text = replace_typical_misspell(text)
    
    #Replace typical groupings
    text = replace_typical_groupings(text)
    
#     #Stem words
#     stemmer = SnowballStemmer('english')
#     text = ' '.join([stemmer.stem(word) for word in text.split(' ')])
    
    return text

In [45]:
train_data = pd.read_csv('../input/train.csv', index_col = 0)
train_data['question_text'] = train_data["question_text"].apply(lambda x: clean(x))

In [43]:
test_data = pd.read_csv('../input/test.csv', index_col = 0)
test_data['question_text'] = test_data["question_text"].apply(lambda x: clean(x))

In [50]:
data_downsample = data_prep_training(train_data, .9, .1)
data_nodownsample = data_prep_training_nodownsample(train_data, .9, .1)

In [85]:
train_data_withgroupings = pd.read_csv('../input/train.csv', index_col = 0)
train_data_withgroupings['question_text'] = train_data_withgroupings["question_text"].apply(lambda x: clean(x))

In [86]:
data_withgroupings_nodownsample = data_prep_training_nodownsample(train_data_withgroupings, .9, .1)
data_withgroupings_nodownsample_results = model_vectorize(data_withgroupings_nodownsample, CountVectorizer, True, (1,4), None, LogisticRegression())
print(data_withgroupings_nodownsample_results)

(0.63941672885929157, 0.16270017933393438)


In [89]:
train_data_withgroupings_nostem = pd.read_csv('../input/train.csv', index_col = 0)
train_data_withgroupings_nostem['question_text'] = train_data_withgroupings_nostem["question_text"].apply(lambda x: clean(x))
data_withgroupings_nodownsample_nostem = data_prep_training_nodownsample(train_data_withgroupings_nostem, .9, .1)
data_withgroupings_nodownsample_no_stem_results = model_vectorize(data_withgroupings_nodownsample_nostem, CountVectorizer, True, (1,4), None, LogisticRegression())


NameError: name 'data_withgroupings_nodownsample_nostem_results' is not defined

In [90]:
data_withgroupings_nodownsample_no_stem_results

(0.64254803531244598, 0.15558298283311578)

In [91]:
#create models
dataset = [data_prep_training_nodownsample(train_data_withgroupings, .9, .1), 
           data_prep_training_nodownsample(train_data_withgroupings_nostem, .9, .1)]
vectorizer_type = [CountVectorizer, TfidfVectorizer]
binary_type = [True, False]
ngram = [(1,2), (1,3), (1,4)]
stop_word = [None, stop]
model_type = [LogisticRegression(), BernoulliNB()]

model_initialize = []
for h in dataset:
    for i in vectorizer_type:
        for j in binary_type:
            for k in ngram:
                for l in model_type:
                    for m in stop_word:
                        model_initialize.append(model_vectorize(h, i, j, k, m, l))
                    
#create labels
dataset_label = ['No downsample Cleaned', 'No Downsample Cleaned No Stem']
vectorizer_type_label = ['CountV', 'TFIDV']
binary_type_label = ['T', 'F']
ngram_label = [(1,2), (1,3), (1,4)]
stop_word_label = ['None', 'english']
model_type_label = ['LR', 'NB']

labels = []
for h in dataset_label:    
    for i in vectorizer_type_label:
        for j in binary_type_label:
            for k in ngram_label:
                for l in model_type_label:
                    for m in stop_word_label:
                        label = '%s %s %s %s %s %s' %(h, i, j, k, m, l)
                        labels.append(label)
                        
fscores = []
thresholds = []
for i in model_initialize:
    fscores.append(i[0])
    thresholds.append(i[1])
metrics = pd.DataFrame({'label': labels,'fscore': fscores, 'threshold': thresholds})
metrics.sort_values(by = ['fscore'], ascending=False)

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in true_divide


,fscore,label,threshold
52,0.646017,"No Downsample Cleaned No Stem CountV T (1, 3) None LR",1.837884e-01
56,0.645811,"No Downsample Cleaned No Stem CountV T (1, 4) None LR",1.550502e-01
68,0.644574,"No Downsample Cleaned No Stem CountV F (1, 4) None LR",1.536640e-01
64,0.643998,"No Downsample Cleaned No Stem CountV F (1, 3) None LR",1.756927e-01
8,0.639218,"No downsample Cleaned CountV T (1, 4) None LR",1.713877e-01
4,0.638754,"No downsample Cleaned CountV T (1, 3) None LR",1.909556e-01
20,0.638194,"No downsample Cleaned CountV F (1, 4) None LR",1.515478e-01
48,0.637048,"No Downsample Cleaned No Stem CountV T (1, 2) None LR",2.256010e-01
16,0.636419,"No downsample Cleaned CountV F (1, 3) None LR",1.530486e-01
60,0.634470,"No Downsample Cleaned No Stem CountV F (1, 2) None LR",2.001840e-01


In [53]:
#create models
dataset = [data_downsample, data_nodownsample]
vectorizer_type = [CountVectorizer, TfidfVectorizer]
binary_type = [True, False]
ngram = [(1,2), (1,3), (1,4)]
stop_word = [None, stop]
model_type = [LogisticRegression(), BernoulliNB()]

model_initialize = []
for h in dataset:
    for i in vectorizer_type:
        for j in binary_type:
            for k in ngram:
                for l in model_type:
                    for m in stop_word:
                        model_initialize.append(model_vectorize(h, i, j, k, m, l))
                    
#create labels
dataset_label = ['Downsample Cleaned', 'No Downsample Cleaned']
vectorizer_type_label = ['CountV', 'TFIDV']
binary_type_label = ['T', 'F']
ngram_label = [(1,2), (1,3), (1,4)]
stop_word_label = ['None', 'english']
model_type_label = ['LR', 'NB']

labels = []
for h in dataset_label:    
    for i in vectorizer_type_label:
        for j in binary_type_label:
            for k in ngram_label:
                for l in model_type_label:
                    for m in stop_word_label:
                        label = '%s %s %s %s %s %s' %(h, i, j, k, m, l)
                        labels.append(label)
                        
fscores = []
thresholds = []
for i in model_initialize:
    fscores.append(i[0])
    thresholds.append(i[1])
metrics = pd.DataFrame({'label': labels,'fscore': fscores, 'threshold': thresholds})
metrics.sort_values(by = ['fscore'], ascending=False)

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in true_divide


,fscore,label,threshold
56,0.651581,"No Downsample Cleaned CountV T (1, 4) None LR",1.636935e-01
68,0.650032,"No Downsample Cleaned CountV F (1, 4) None LR",1.679055e-01
52,0.649858,"No Downsample Cleaned CountV T (1, 3) None LR",1.787951e-01
64,0.648345,"No Downsample Cleaned CountV F (1, 3) None LR",1.764204e-01
48,0.644638,"No Downsample Cleaned CountV T (1, 2) None LR",2.395410e-01
60,0.641428,"No Downsample Cleaned CountV F (1, 2) None LR",2.359062e-01
84,0.640102,"No Downsample Cleaned TFIDV F (1, 2) None LR",2.595899e-01
72,0.638673,"No Downsample Cleaned TFIDV T (1, 2) None LR",2.855823e-01
53,0.633056,"No Downsample Cleaned CountV T (1, 3) english LR",1.777751e-01
49,0.632324,"No Downsample Cleaned CountV T (1, 2) english LR",2.184303e-01


In [55]:
train_bestmodel = train_data.copy()
train_bestmodel['question_text'] = train_bestmodel["question_text"].apply(lambda x: clean(x))
test_bestmodel = test_data.copy()
test_bestmodel['question_text'] = test_bestmodel["question_text"].apply(lambda x: clean(x))

test_results = run_model(train_bestmodel, test_bestmodel, CountVectorizer, True, (1,4), None, LogisticRegression(), .1636935)

In [90]:
test_results = pd.DataFrame()
test_results['index'] = test_data.index
test_results['prediction_percent'] = test[0]
test_results['prediction'] = test[1]

In [94]:
results = test_results.drop('prediction_percent', 1)
results.to_csv('submission.csv', index = False)

In [80]:
count = 0
ex = []
for text in train_data.question_text:
    if 'gen x' in text:
        count = count +1
        ex.append(text)
count

9

In [79]:
ex

['what issu can millenni and gen x agre upon',
 'whi do gen x peopl react more with insensit toward their mentallyil children',
 'a gaseous hydrogen x was burnt in excess of oxygen a #### dm³ sampl of x at stp game ###g of co# how mani catom are there in one molecul of x',
 'what is like for a gen x date a millenni',
 'what is the composit of gen x and gen y in the popul of bangalor',
 'what is the biggest mistak the indian gen x did',
 'how can i switch from be gen x to gen y',
 'what is the volkswagen xl# car about',
 'whi are indian men gen x and earlier so egoist']